In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [5]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')


In [6]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)

In [8]:
# splitting the data into training and testing
#for testing
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev= X_dev/255

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train/255
_,m_train= X_train.shape


In [9]:
def init_params():
    w1=np.random.rand(10,784)-0.5
    b1=np.random.rand(10,1)-0.5
    w2=np.random.rand(10,10)-0.5
    b2=np.random.rand(10,1)-0.5
    return w1,b1,w2,b2

In [10]:
def ReLU(z):
    return np.maximum(0,z)

In [11]:
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

In [12]:
def one_hot(z):
    one_hot_z=np.zeros((z.size,z.max()+1))
    one_hot_z[np.arange(z.size),z]=1
    one_hot_z=one_hot_z.T
    return one_hot_z

In [13]:
def derivative_ReLU(z):
    return z>0

In [16]:
def forward_prop(w1,b1,w2,b2,x):
    z1=w1.dot(x)+b1
    a1=ReLU(z1)
    z2=w2.dot(a1)+b2
    a2=softmax(z2)
    return z1,a1,z2,a2

In [17]:
def back_prop(z1,a1,z2,a2,w2,x,y):
    m=y.size
    one_hot_Y=one_hot(y)
    dz2=a2-one_hot_Y
    dw2=1/m * dz2.dot(a1.T)
    db2=1/m * np.sum(dz2)
    dz1 = w2.T.dot(dz2)*derivative_ReLU(z1)
    dw1=1/m * dz1.dot(x.T)
    db1=1/m * np.sum(dz1)
    return dw1,db1,dw2,db2

In [18]:
def update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha):
    w1=w1-alpha*dw1
    w2=w2-alpha*dw2
    b1=b1-alpha*db1
    b2=b2-alpha*db2
    return w1,b1,w2,b2

In [20]:
def get_predictions(a2):
    return np.argmax(a2,0)
    


In [21]:
def get_accuracy(predictions,y):
    print(predictions,y)
    return np.sum(predictions==y)/y.size


In [22]:
def gradient_descent(x,y,it,alpha):
    w1,b1,w2,b2=init_params()
    for i in range(it):
        z1,a1,z2,a2=forward_prop(w1,b1,w2,b2,x)
        dw1,db1,dw2,db2=back_prop(z1,a1,z2,a2,w2,x,y)
        w1,b1,w2,b2=update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha)
        if i%10==0:
            print("Iteration",i)
            print("Accuracy", get_accuracy(get_predictions(a2),y))
    return w1,b1,w2,b2

w1,b1,w2,b2=gradient_descent(X_train,Y_train,500,0.1)

Iteration 0
[1 0 9 ... 4 1 4] [2 5 3 ... 6 1 1]
Accuracy 0.14265853658536584
Iteration 10
[1 0 3 ... 9 1 4] [2 5 3 ... 6 1 1]
Accuracy 0.26607317073170733
Iteration 20
[1 3 2 ... 6 1 4] [2 5 3 ... 6 1 1]
Accuracy 0.3211219512195122
Iteration 30
[1 3 2 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.38902439024390245
Iteration 40
[5 8 3 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.4629268292682927
Iteration 50
[5 8 3 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.5114634146341464
Iteration 60
[5 8 3 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.5478536585365854
Iteration 70
[5 8 3 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.577780487804878
Iteration 80
[2 8 3 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.6045121951219512
Iteration 90
[2 8 3 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.6249024390243902
Iteration 100
[2 8 3 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.6439024390243903
Iteration 110
[2 8 3 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.6615121951219513
Iteration 120
[2 8 3 ... 6 1 1] [2 5 3 ... 6 1 1]
Accuracy 0.677707317073

In [23]:
def make_predictions(x,w1,b1,w2,b2):
    _,_,_,a2=forward_prop(w1,b1,w2,b2,x)
    predictions=get_predictions(a2)
    return predictions

In [26]:
def test_predictions(index,w1,b1,w2,b2):
    current_image=X_train[:,index,None]
    prediction=make_predictions(X_train[:,index,None],w1,b1,w2,b2)
    label=Y_train[index]
    print("Prediction : ",prediction)
    print("Label : ",label)
    current_image=current_image.reshape((28,28))*255
    plt.gray()
    plt.imshow(current_image,interpolation='nearest')
    plt.show()

In [25]:
dev_predictions=make_predictions(X_dev,w1,b1,w2,b2)
print("Accuracy - ",get_accuracy(dev_predictions,Y_dev))

[1 2 7 0 6 8 2 1 2 0 6 7 3 0 3 0 8 4 3 6 7 9 5 2 6 4 8 5 8 9 7 9 0 9 0 2 3
 6 5 7 7 6 1 1 6 1 9 0 4 1 2 5 3 2 1 6 3 9 8 5 3 9 9 4 0 1 2 1 9 2 1 4 6 1
 4 4 8 8 3 9 5 4 8 0 0 4 1 3 0 3 4 2 7 2 8 7 9 4 6 9 2 4 1 9 0 3 0 5 7 9 3
 3 9 3 6 5 6 0 4 6 3 0 1 8 9 7 2 5 6 4 6 7 6 6 0 6 2 6 2 3 1 6 5 5 0 4 1 8
 9 6 8 8 7 6 3 3 1 2 6 6 6 7 1 9 4 9 0 3 2 6 2 2 5 2 8 1 4 3 5 1 5 0 1 0 0
 7 5 1 5 4 4 0 3 7 3 9 0 7 4 4 2 3 8 3 8 3 4 5 4 7 4 6 2 3 8 2 5 5 1 9 1 8
 3 6 0 9 2 2 1 6 6 6 6 7 8 7 7 7 4 6 5 9 0 7 8 2 3 7 1 5 9 1 3 8 6 1 5 3 2
 2 6 0 1 0 5 2 6 3 0 4 7 9 3 2 3 9 6 1 1 4 5 8 9 2 4 6 6 6 7 9 4 8 5 3 3 1
 6 1 1 9 4 1 0 5 5 0 9 9 7 4 8 6 5 7 8 3 8 8 3 9 9 7 6 9 3 4 6 7 9 9 1 4 6
 2 4 4 2 0 6 6 7 5 2 3 6 6 6 4 2 6 4 4 5 4 4 0 5 0 3 2 7 6 9 2 3 7 4 2 8 1
 2 7 4 6 1 6 7 8 4 6 9 6 7 9 4 3 6 2 2 1 1 1 7 2 5 9 4 3 1 3 7 2 6 2 2 2 8
 8 8 9 1 8 0 4 6 5 1 7 6 0 4 8 4 4 8 1 0 8 2 2 3 8 9 5 1 4 4 2 0 1 0 7 7 2
 1 6 8 8 9 9 0 6 7 5 8 7 1 3 2 0 9 7 9 5 2 8 9 3 1 6 1 0 9 2 6 2 2 5 0 1 2
 7 4 6 9 9 4 4 1 0 2 0 8 